# Tarea 1

Integrantes: Joaquin Garay y Lucas Ribas

Implementar la clase `OisCashflow`.

## Configuración Inicial



In [2]:
from finrisk import QC_Financial_3 as Qcf
from dataclasses import dataclass
from enum import Enum
import pandas as pd

# Modificado por AD
import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

## Importación objeto `qcf.time_series`

In [3]:
df = pd.read_excel('../data/SOFR-10012019-10292020.xls', sheet_name = 'nice format')

df['DATE'] = df['DATE'].astype(str).str[:10]
df['DATE'] = pd.to_datetime(df['DATE'], format = '%Y-%m-%d')

df['DATE'] = df['DATE'].dt.date

df = df.sort_values(by='DATE')

df.columns = df.columns.str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

df.head()

,DATE,_BENCHMARK_NAME_,RATE\nPERCENT
269,2019-10-01,SOFR,1.88
268,2019-10-02,SOFR,1.85
267,2019-10-03,SOFR,1.84
266,2019-10-04,SOFR,1.82
265,2019-10-07,SOFR,1.83


Luego se modifica las fechas a tipo datetime


In [4]:
df['DATE'] = df['DATE'].astype(str).str[:10]
df['DATE'] = pd.to_datetime(df['DATE'], format = '%Y-%m-%d')

df['DATE'] = df['DATE'].dt.date

df = df.sort_values(by='DATE')

df.columns = df.columns.str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

df.head()

,DATE,_BENCHMARK_NAME_,RATE\nPERCENT
269,2019-10-01,SOFR,1.88
268,2019-10-02,SOFR,1.85
267,2019-10-03,SOFR,1.84
266,2019-10-04,SOFR,1.82
265,2019-10-07,SOFR,1.83


De fechas datetime se transforman a fecha Qcf.Date

In [5]:
df['FECHA'] = df.apply(
    lambda row: Qcf.QCDate(
    row ['DATE'].day,
    row ['DATE'].month,
    row ['DATE'].year),
    axis = 1
)

df['TASAS'] = df.apply(
    lambda row: row ['RATE\nPERCENT']/100,
    axis = 1
)

del df['DATE']
del df["_BENCHMARK_NAME_"]
del df["RATE\nPERCENT"]

In [6]:
df.head()

,FECHA,TASAS
269,1-10-2019,0.0188
268,2-10-2019,0.0185
267,3-10-2019,0.0184
266,4-10-2019,0.0182
265,7-10-2019,0.0183


Luego, con un dataframe donde las fechas estan en tipo Qcf.date, se pasa la data a un Qcf.time_series

In [7]:
ts = Qcf.time_series()

for i,row in enumerate(df.iterrows()):
    ts[df['FECHA'].iloc[i]] = df['TASAS'].iloc[i]

## Calendario

In [8]:
class BusCal(Enum):
    NY = 1
    SCL = 2

In [9]:
def get_cal(code: BusCal) -> Qcf.BusinessCalendar:
    """
    """
    if code == BusCal.NY:
        cal = Qcf.BusinessCalendar(Qcf.QCDate(1, 1, 2020), 20)
        for agno in range(2020, 2071):
            f = Qcf.QCDate(12, 10, agno)
            if f.week_day() == Qcf.WeekDay.SAT:
                cal.add_holiday(Qcf.QCDate(14, 10, agno))
            elif f.week_day() == Qcf.WeekDay.SUN:
                cal.add_holiday(Qcf.QCDate(13, 10, agno))
            elif f.week_day() == Qcf.WeekDay.MON:
                cal.add_holiday(Qcf.QCDate(12, 10, agno))
            elif f.week_day() == Qcf.WeekDay.TUE:
                cal.add_holiday(Qcf.QCDate(11, 10, agno))
            elif f.week_day() == Qcf.WeekDay.WED:
                cal.add_holiday(Qcf.QCDate(10, 10, agno))
            elif f.week_day() == Qcf.WeekDay.THU:
                cal.add_holiday(Qcf.QCDate(9, 10, agno))
            else:
                cal.add_holiday(Qcf.QCDate(8, 10, agno))
        cal.add_holiday(Qcf.QCDate(15, 2, 2021))
        
    return cal

## Importación objeto `qcf.ZeroCouponCurve`

In [10]:
frmt = {
    'tasa': '{:.6%}',
    'df': '{:.6%}',
    'valor_tasa': '{:.4%}',
    'spread': '{:.4%}',
    'nominal': '{:,.2f}',
    'interes': '{:,.2f}',
    'amortizacion': '{:,.2f}',
    'flujo': '{:,.2f}',
}

df_curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')
zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df_curva)
df_curva.head().style.format(frmt)

,plazo,tasa,df
0,1,0.081111%,99.999778%
1,7,0.084051%,99.998388%
2,14,0.077967%,99.997010%
3,21,0.077358%,99.995549%
4,33,0.078067%,99.992942%


## Clase `OisCashflow`

La clase `OisCashflow` es una dataclass. La tarea consiste en:


1.   Implementar los 3 métodos de la clase que estan con `pass`
2.   Implementar la función `set_expected_rate`
3.   Implementar la función `present_value`



In [11]:
@dataclass # syntactic sugar
class OisCashflow:
    start_date: Qcf.QCDate
    end_date: Qcf.QCDate
    settlement_date: Qcf.QCDate
    notional: float
    currency: Qcf.QCCurrency
    amortization: float
    amort_is_cashflow: bool
    interest_rate: Qcf.QCInterestRate
    on_index: Qcf.InterestRateIndex
    spread: float
    gearing: float
        
    def get_accrued_rate(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
        """
        Calcula la tasa equivalente desde `start_date` a `accrual_date`. La tasa equivalente
        se calcula como:
        
        (P - 1) * 360 / (accrual_date - start_date)
        
        donde P es el producto de los factores de capitalización de todas las tasas overnight
        entre `start_date` y `accrual_date`. Los valores de esas tasas deben estar almacenados
        en la variable `fixings`.

        Se considera tasas lineal actual 360. Se considera que los intereses devengados hasta
        un dia antes del accrual_date que termina de devengar en accr
        """

        P = 1
        
        for j, date in enumerate(Qcf.time_series_dates(fixings)):
            
            if date == self.start_date:
                
                for i, value in enumerate(Qcf.time_series_values(fixings)[j:]):
            
                    qc_fec_ini = Qcf.time_series_dates(fixings)[j+i]
            
                    qc_fec_fin = Qcf.time_series_dates(fixings)[j+i+1]
            
                    qc_tasa = Qcf.QCInterestRate(value, Qcf.QCAct360(), Qcf.QCLinearWf())
            
                    P = P * qc_tasa.wf(qc_fec_ini, qc_fec_fin)
            
                    if accrual_date == qc_fec_fin:
                        
                        break
             
        
        # Número de días entre la fecha inicial y la fecha final
        dias = self.start_date.day_diff(accrual_date)
        
        accrued_rate = (P - 1) * 360 / dias
        
        return accrued_rate
    
    def get_accrued_interest(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
        """
        Calcula el interés devengado desde `start_date` a `accrual_date` utilizando la tasa equivalente
        que se calcula con el método anterior.
        
        Los valores de las  tasas overnight deben estar almacenados en la variable `fixings`.
        """
        
        tasaeq = self.get_accrued_rate(accrual_date, fixings)
        
        return tasaeq * self.notional

    
    def amount(self, fixings: Qcf.time_series) -> float:
        """
        Calcula el flujo total al vencimiento (amortización más intereses devengados hasta end_date).
        
        Los valores de las  tasas overnight deben estar almacenados en la variable `fixings`.
        """
        flujo = self.get_accrued_interest(self.end_date, fixings) + self.amortization
        
        return flujo

Construcción de un objeto `Qcf.InterestRateIndex`

In [12]:
codigo = 'SOFR'
tasa_on = Qcf.QCInterestRate(.0, Qcf.QCAct360(), Qcf.QCLinearWf())
fixing_lag = Qcf.Tenor('0d')
tenor = Qcf.Tenor('1d')
fixing_calendar = get_cal(BusCal.NY)     
settlement_calendar = fixing_calendar  
sofr = Qcf.InterestRateIndex(
    codigo,
    tasa_on,
    fixing_lag,
    tenor,
    fixing_calendar,
    settlement_calendar,
    Qcf.QCUSD()
)

Construcción de una instancia `OisCashflow`

In [13]:
ois = OisCashflow(
    Qcf.build_qcdate_from_string('2019-10-01'), # start_date: Dia 1 del fixings.
    Qcf.build_qcdate_from_string('2021-10-01'), # end date: Algun dia posterior al termino del fixing.
    Qcf.build_qcdate_from_string('2019-10-01'), # settlement day: Igual que start_date por simplicidad.
    10e6,
    Qcf.QCUSD(),
    10e6,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1
)

## Funciones:

### Set_expected_rate

In [14]:
def set_expected_rate(
        val_date: Qcf.QCDate,
        ois_cashflow: OisCashflow,
        zcc: Qcf.ZeroCouponCurve,
        fixings: Qcf.time_series) -> None:
    """
    Se consideran dos casos en relacion a la fecha val_date 
    """
    if (val_date <= ois_cashflow.start_date) :
        
        # val_date < start_date < end_date
        
        plazo1 = val_date.day_diff(ois_cashflow.start_date)
        plazo2 = val_date.day_diff(ois_cashflow.end_date)
        
        expected_rate = zcc.get_forward_rate_with_rate(
            Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
            plazo1,
            plazo2)
        
    else:
        
        # start_date < val_date < end_date 
        
        tasa1 = ois_cashflow.get_accrued_rate(val_date, fixings) #Desde start_date hasta val_date. Conocido -> fixings
    
        plazo1 = ois_cashflow.start_date.day_diff(val_date)
    
        WF1 = (tasa1 * plazo1 / 360) + 1 #Considerando convencion de tasas lineales.
    
        plazo2 = val_date.day_diff(ois_cashflow.end_date)
    
        tasa2 = zcc.get_forward_rate_with_rate(
                Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
                0,
                plazo2) #Desde val_date hasta end_date. Desconocido -> zcc
    
        WF2 = (tasa2 * plazo2 / 360) + 1 #Considerando convencion de tasas lineales.
    
        WF = WF1 * WF2
        
        expected_rate = (WF - 1) * 360 / (plazo1 + plazo2)
    
    
    ois_cashflow.interest_rate.set_value(expected_rate)
        
    print(f'Sofr esperada: {expected_rate:.6%}')

Comprobación

In [15]:
val_date = Qcf.build_qcdate_from_string('2019-08-01')
set_expected_rate(val_date, ois, zcc, ts)

print(ois.interest_rate.get_value())

Sofr esperada: 0.057539%
0.0005753868227933123


### present_value

In [16]:
def present_value(val_date: Qcf.QCDate, ois_cashflow: OisCashflow, zcc: Qcf.ZeroCouponCurve) -> float:
    """
    Esta función opera de la misma forma que la análoga función de `QC_Financial_3`.
    
    Comentario: Función construida para el caso t0 < t1 < t2, ya que si t1 < t0 < t2, se estaría trayendo a valor presente
    una parte de los flujos que en realidad está en el pasado.
    
    """
    tasa_objeto = ois_cashflow.interest_rate.get_value()
    
    plazo1 = val_date.day_diff(ois_cashflow.start_date)
    plazo2 = val_date.day_diff(ois_cashflow.end_date)
        
    expected_discount_rate = zcc.get_forward_rate_with_rate(
            Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
            plazo1,
            plazo2)
    
    dias = ois_cashflow.start_date.day_diff(ois_cashflow.end_date)
    
    df = 1/(expected_discount_rate*(dias/360) + 1)
    
    flujo = ois_cashflow.amortization + ois_cashflow.notional*tasa_objeto*(dias/360)
    
    pv = flujo*df
    
    return pv

Comprobación

In [17]:
pv = present_value(val_date, ois, zcc)
print(pv)

10000000.0


## Tests

In [18]:
import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

In [19]:
amount_tol = 10000
rate_tol = .001

In [20]:
exitos = 0

In [21]:
def suma_exito(resultado, check, tipo):
    global exitos
    if resultado is None:
        return
    else:
        print('Suma medio punto por obtener resultado.')
        exitos += .5
        if tipo == 'monto':
            if abs(resultado - check) < amount_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1
        else:
            if abs(resultado - check) < rate_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1

Construcción de una instancia de `OisCashflow`.

In [23]:
ois = OisCashflow(
    Qcf.QCDate(1, 10, 2019),
    Qcf.QCDate(1, 10, 2020),
    Qcf.QCDate(1, 10, 2020),
    10000000,
    Qcf.QCUSD(),
    1000000,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1
)

En la siguiente variable `exitos` se registra cuantos tests se superan con éxito.

### Objeto `fixings`

In [24]:
df_fixings = pd.read_excel('../data/SOFR-10012019-10292020.xls', sheet_name='nice format')
df_fixings.columns = ['fecha', 'nombre', 'valor']
df_fixings['valor'] /= 100

In [25]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


In [26]:
fixings = Qcf.time_series()
for row in df_fixings.itertuples():
    fixings[Qcf.build_qcdate_from_string(row.fecha)] = row.valor

In [27]:
fixings[Qcf.QCDate(13, 1, 2020)]

0.0154

### `OisCashflow.accrued_rate`

In [28]:
accrued_rate = ois.get_accrued_rate(Qcf.QCDate(15, 6, 2020), fixings)
print(f'accrued rate: {accrued_rate:.8%}')

accrued rate: 1.04766527%


Se verificará el cálculo usando `df_fixings`.

In [29]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


Se agrega la columna `next_date`, servirá para calcular el factor de capitalización de cada tasa.

In [30]:
def next_date(fecha:str, calendario: Qcf.BusinessCalendar) -> str:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    next_fecha = calendario.shift(qfecha, 1)
    return next_fecha.description(False)

In [31]:
df_fixings['next_fecha'] = df_fixings.apply(
    lambda row: next_date(row['fecha'], settlement_calendar),
    axis=1
)

In [32]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha
0,2020-10-28,SOFR,0.0800%,2020-10-29
1,2020-10-27,SOFR,0.0900%,2020-10-28
2,2020-10-26,SOFR,0.0900%,2020-10-27
3,2020-10-23,SOFR,0.0800%,2020-10-26
4,2020-10-22,SOFR,0.0700%,2020-10-23


Se calcula ahora el factor de capitalización.

In [33]:
def factor_cap(fecha: str, next_fecha: str, valor: float) -> float:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    qnext_fecha = Qcf.build_qcdate_from_string(next_fecha)
    int_rate = Qcf.QCInterestRate(valor, Qcf.QCAct360(), Qcf.QCLinearWf())
    return int_rate.wf(qfecha, qnext_fecha)

In [34]:
df_fixings['factor_capitalizacion'] = df_fixings.apply(
    lambda row: factor_cap(row['fecha'], row['next_fecha'], row['valor']),
    axis=1
)

In [35]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha,factor_capitalizacion
0,2020-10-28,SOFR,0.0800%,2020-10-29,1.000002
1,2020-10-27,SOFR,0.0900%,2020-10-28,1.000002
2,2020-10-26,SOFR,0.0900%,2020-10-27,1.000002
3,2020-10-23,SOFR,0.0800%,2020-10-26,1.000007
4,2020-10-22,SOFR,0.0700%,2020-10-23,1.000002


Veamos donde están en `df_fixings` los valores para `start_date` de `ois` y la fecha 15-6-2020 y calculamos el producto de los factores de capitalización desde esa fecha hasta la última fecha de `df_flujos` (que es el primer registro). Luego, el cociente de los dos factores nos dará el factor entre ambas fechas. Finalmente, con ese factor, se calculará `accrued_rate`.

In [36]:
start_date = ois.start_date.description(False)
df_fixings[df_fixings.fecha == start_date]

,fecha,nombre,valor,next_fecha,factor_capitalizacion
269,2019-10-01,SOFR,0.0188,2019-10-02,1.000052


In [37]:
factor_largo = df_fixings.iloc[:270]['factor_capitalizacion'].prod()
print(factor_largo)
qstart_date = Qcf.build_qcdate_from_string('2019-10-01')

1.0075715527341267


In [38]:
df_fixings[df_fixings.fecha == '2020-06-15']

,fecha,nombre,valor,next_fecha,factor_capitalizacion
94,2020-06-15,SOFR,0.0009,2020-06-16,1.000002


In [39]:
factor_corto = df_fixings.iloc[:95]['factor_capitalizacion'].prod()
qaccrued_date = Qcf.build_qcdate_from_string('2020-06-15')

In [40]:
dias = qstart_date.day_diff(qaccrued_date)
check_accrued_rate = (factor_largo / factor_corto - 1.0) * 360.0 / dias
print(f'La diferencia es: {accrued_rate - check_accrued_rate:.8%}')

La diferencia es: 0.03775213%


In [41]:
suma_exito(accrued_rate, check_accrued_rate, 'tasa')

Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


### `OisCashflow.accrued_interest`

In [42]:
accrued_interest = ois.get_accrued_interest(Qcf.QCDate(15, 6, 2020), fixings)
print(f'accrued interest: {accrued_interest:,.2f}')

accrued interest: 104,766.53


In [43]:
check_accrued_interest = ois.notional * accrued_rate * dias / 360.0
print(f'La diferencia es: {accrued_interest - check_accrued_interest:,.6f}')

La diferencia es: 29,683.849178


In [44]:
suma_exito(accrued_interest, check_accrued_interest, 'monto')

Suma medio punto por obtener resultado.


### `OisCashflow.amount`

In [45]:
amount = ois.amount(fixings)
print(f'amount: {amount:,.2f}')

amount: 1,076,574.66


In [46]:
end_date = ois.end_date.description(False)
print(f'end date: {end_date}')
df_fixings[df_fixings.fecha == end_date]

end date: 2020-10-01


,fecha,nombre,valor,next_fecha,factor_capitalizacion
18,2020-10-01,SOFR,0.0008,2020-10-02,1.000002


In [47]:
factor_corto_2 = df_fixings.iloc[:19]['factor_capitalizacion'].prod()
dias_2 = qstart_date.day_diff(ois.end_date)
accrued_rate_amount = (factor_largo / factor_corto_2- 1.0) * 360.0 / dias_2
check_amount = ois.notional * accrued_rate_amount * dias_2 / 360.0 + ois.amortization
print(f'La diferencia es: {amount - check_amount:.6f}')

La diferencia es: 1547.609929


In [48]:
suma_exito(amount, check_amount, 'monto')

Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


### `set_expected_rate`

Comenzamos cargando los valores de la curva cero cupón que se utilizará para hacer pruebas de `set_expected_rate` y `present_value`.

In [49]:
df_curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')

In [50]:
df_curva.head().style.format({'tasa': '{:.4%}', 'df': '{:.6%}'})

,plazo,tasa,df
0,1,0.0811%,99.999778%
1,7,0.0841%,99.998388%
2,14,0.0780%,99.997010%
3,21,0.0774%,99.995549%
4,33,0.0781%,99.992942%


Con la data se construye un objeto de tipo `Qcf.ZeroCouponCurve`.

In [51]:
zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df_curva)

Comienzan los tests.

In [52]:
print(f'Fecha inicial flujo: {ois.start_date.description(False)}')
print(f'Fecha final flujo: {ois.end_date.description(False)}')

Fecha inicial flujo: 2019-10-01
Fecha final flujo: 2020-10-01


In [54]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Sofr esperada: 0.061213%
Tasa esperada es: 0.06121277%
Check tasa esperada es: 0.06121277%
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


In [55]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Sofr esperada: 0.069243%
Tasa esperada es: 0.06924275%
Check tasa esperada es: 0.06924275%
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


In [56]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = ois.start_date.day_diff(val_date)
p2 = val_date.day_diff(ois.end_date)
wf1 = 1 + ois.get_accrued_interest(val_date, fixings) / ois.notional
wf2 = 1 / zcc.get_discount_factor_at(p2)
check = (wf1 * wf2 - 1) * 360 / (p1 + p2)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Sofr esperada: 0.221664%
Tasa esperada es: 0.22166384%
Check tasa esperada es: 1.89539287%
Suma medio punto por obtener resultado.


In [57]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(val_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Sofr esperada: nan%
Tasa esperada es: nan%
Check tasa esperada es: 0.76574655%
Suma medio punto por obtener resultado.


In [58]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(ois.end_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Sofr esperada: 0.769271%
Tasa esperada es: 0.76927110%
Check tasa esperada es: 0.76574655%
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


### `present_value`

In [60]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Sofr esperada: 0.061213%
Valor presente es: 1,005,597.4846
Amount: 1,076,574.6554
Df: 99.911652%
VP check es: 1,075,623.5257
Suma medio punto por obtener resultado.


In [61]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Sofr esperada: 0.069243%
Valor presente es: 1,006,331.2547
Amount: 1,076,574.6554
Df: 99.929653%
VP check es: 1,075,817.3144
Suma medio punto por obtener resultado.


In [62]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Sofr esperada: 0.221664%
Valor presente es: 1,021,871.3744
Amount: 1,076,574.6554
Df: 99.935019%
VP check es: 1,075,875.0913
Suma medio punto por obtener resultado.


In [63]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Sofr esperada: nan%
Valor presente es: nan
Amount: 1,076,574.6554
Df: 0.000000%
VP check es: 0.0000
Suma medio punto por obtener resultado.


In [64]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Sofr esperada: 0.769271%
Valor presente es: 1,078,209.2285
Amount: 1,076,574.6554
Df: 0.000000%
VP check es: 0.0000
Suma medio punto por obtener resultado.


In [65]:
print(f'Éxitos totales: {exitos}')

Éxitos totales: 11.5
